In [3]:
import torch
import os
from dotenv import load_dotenv
import tiktoken
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [4]:
load_dotenv('./.env')

True

#### Leitura do arquivo e criação do splitter'

In [5]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [6]:
chunks = text_splitter.create_documents([clt])

In [7]:
# Função para calcular o valor do embedding usando o ada-002 da OpenAI
def embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    total_em_dolar = total_tokens / 1000 * 0.0001
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_em_dolar:.6f}')
    print(f'Custo de Embedding em R$: {total_em_dolar * 4.95:.6f}')

In [8]:
embedding_cost(chunks)

Total de tokens: 818
Custo de Embedding em USD: 0.000082
Custo de Embedding em R$: 0.000405


In [9]:
embeddings = OpenAIEmbeddings()

### Pinecone

Pinecone é um serviço de banco de dados vetorizado projetado para buscas por similaridade em larga escala. Ele permite que você armazene e pesquise vetores de alta dimensão, que são representações numéricas de dados complexos, como textos, imagens, e mais.

In [10]:
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

In [11]:
indexes = pinecone.list_indexes()
index_name = 'natgpt'

In [12]:
indexes = pinecone.list_indexes()
for i in indexes:
    print('Index encontrada e apagada: ' + i)
    pinecone.delete_index(i)

Index encontrada e apagada: natgpt


In [13]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    print('Index '+index_name+' criado')

Index natgpt criado


In [14]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

#### Busca por similaridade

A busca por similaridade, no contexto de Pinecone, envolve a comparação de vetores numéricos para encontrar aqueles que são mais próximos entre si em termos de distância em um espaço vetorial. Essa "proximidade" pode ser calculada usando métricas como distância euclidiana, similaridade de cosseno, entre outras. Quanto menor a distância ou maior a similaridade, mais semelhantes são os itens.

<div align="center">
    <img src="images/similarity.png" width="50%">
</div>

In [15]:
query = 'O que é o NAT?'
result = vector_store.similarity_search(query, k=3) # Retorna os 3 resultados mais similares
print(result)

[Document(page_content='Prova disso é que no ano de 2019, o NAT atendeu diversas demandas de\nbaixa, média e alta complexidade, em processos e/ou procedimentos judiciais e\nextrajudiciais, além de dar suporte ao Grupo de Atuação Especial de Combate ao\nCrime Organizado – GAECO na condução de operações importantes e de suas\natividades diárias.'), Document(page_content='O Núcleo de Apoio Técnico (NAT) foi instituído no âmbito do Ministério\nPúblico do Estado do Acre por meio do Ato n.º 25, de 13 de setembro de 2012, da\nProcuradoria-Geral de Justiça, com o escopo de prestar apoio de inteligência e\nsegurança institucional, técnico-científico e operacional, por meio de servidores\nhabilitados em áreas de conhecimento específico, aos órgãos de execução e, em\nespecial, ao Grupo de Atuação Especial no Combate ao Crime Organizado\n(GAECO).'), Document(page_content='Posteriormente, o NAT foi regulado pela nova Lei Orgânica do Ministério\nPúblico do Estado do Acre (Lei Complementar n.º 291, d

In [16]:
for r in result:
    print(r.page_content)
    print('-' * 50)

Prova disso é que no ano de 2019, o NAT atendeu diversas demandas de
baixa, média e alta complexidade, em processos e/ou procedimentos judiciais e
extrajudiciais, além de dar suporte ao Grupo de Atuação Especial de Combate ao
Crime Organizado – GAECO na condução de operações importantes e de suas
atividades diárias.
--------------------------------------------------
O Núcleo de Apoio Técnico (NAT) foi instituído no âmbito do Ministério
Público do Estado do Acre por meio do Ato n.º 25, de 13 de setembro de 2012, da
Procuradoria-Geral de Justiça, com o escopo de prestar apoio de inteligência e
segurança institucional, técnico-científico e operacional, por meio de servidores
habilitados em áreas de conhecimento específico, aos órgãos de execução e, em
especial, ao Grupo de Atuação Especial no Combate ao Crime Organizado
(GAECO).
--------------------------------------------------
Posteriormente, o NAT foi regulado pela nova Lei Orgânica do Ministério
Público do Estado do Acre (Lei Compleme

### Conversando com LLM sobre dados

In [17]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":0.5, "max_length":2048},
)

/linux-data/miniconda3/envs/linuxtips-llm/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [18]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [19]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [26]:
# Prompt para evitar que o LLM invente algo caso não saiba a resposta e sempre responda em Português
prompt1 = "Responda em português do Brasil: "
prompt2 = ". Se não souber, useu seu conhecimento. Se não tiver conhecimento, diga apenas 'eu não sei'. Não invente uma resposta"

In [21]:
pergunta = 'Resumidamente, o que é o NAT?'
resp = chain.run(prompt1+pergunta+prompt2)
print(resp)

 Resumidamente, o NAT é um núcleo de apoio técnico, que atende diversas demandas de baixa, média e alta complexidade, em processos e/ouprocedimentos judiciais e extrajudiciais, além de dar suporte ao Grupo de Atuaação Especial de Combate ao Crime Organizado – GAECO na condução de operações importantes e de suas atividades


In [22]:
pergunta = 'Quem são os responsáveis pelo NAT?'
resp = chain.run(prompt1+pergunta+prompt2)
print(resp)

 Marcela Cristina Ozório e Bernardo Fiterman Albano são os responsáveis pelo NAT.


In [23]:
pergunta = 'Qual o site do NAT?'
resp = chain.run(prompt1+pergunta+prompt2)
print(resp)

 O site do NAT é https://nat.mpac.mp.br.


In [28]:
pergunta = 'Quais são as atividades do NAT?'
resp = chain.run(prompt1+pergunta+prompt2)
print(resp)

 As atividades do NAT são diversas e incluem atendimento a demandas de baixa, média e alta complexidade, em processos e/ou procedimentos judiciais e extrajudiciais, além de dar suporte ao Grupo de Atuação Especial de Combate ao Crime Organizado – GAECO na condução de operações importantes e de suas atividades diárias.


In [29]:
pergunta = 'Quando e onde o NAT foi instituído?'
resp = chain.run(prompt1+pergunta+prompt2)
print(resp)

 O NAT foi instituído no âmbito do Ministério Público do Estado do Acre por meio do Ato n.º 25, de 13 de setembro de 2012.
